In [113]:
import pandas as pd
import numpy as np

In [121]:
wightman = pd.read_csv('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/Wightman_et_al_2021_hg37_ldsc.tsv.gz', compression='gzip',sep='\t')
#jansen = pd.read_parquet('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/processed/jansen_et_al_2021_hg37_ldsc.munged.parquet')

In [14]:
jansen_og = pd.read_csv('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/Jansen_et_al_2019_hg37_ldsc.tsv.gz', compression = 'gzip', sep = '\t')

In [122]:
print(wightman.columns)
print(jansen.columns)

Index(['SNP', 'CHR', 'BP', 'A1', 'A2', 'Z', 'P', 'N'], dtype='object')
Index(['SNP', 'CHR', 'BP', 'A1', 'A2', 'Z', 'MAF', 'N'], dtype='object')


### MAF was opposite while A1 and A2 are still the same

In [15]:
print(jansen[jansen.SNP == 'rs12184277'])
print(jansen_og[jansen_og.SNP == 'rs12184277'])


          SNP  CHR      BP A1 A2         Z       MAF       N
1  rs12184277    1  715367  A  G -1.957915  0.041069  360230
          SNP  CHR      BP A1 A2   UNIQID.A1A2         Z        P    NSUM  \
1  rs12184277    1  715367  A  G  1:715367_G_A -1.957915  0.05024  360230   

       NEFF   DIR       FRQ      BETA        SE       N  
1  360230.0  ??+?  0.958931 -0.011624  0.005937  360230  


In [8]:
0.9591931+0.0408


0.9999931

In [123]:
wightman

,SNP,CHR,BP,A1,A2,Z,P,N
0,rs533499096,1,14860,G,T,-1.275874,0.202000,74004
1,rs373918278,1,17765,G,A,0.093886,0.925200,74004
2,rs763779009,1,17767,G,A,-0.426148,0.670000,74004
3,rs199900651,1,48186,T,G,0.913081,0.361200,74004
4,rs555296411,1,108506,C,T,0.269688,0.787400,74004
...,...,...,...,...,...,...,...,...
11991345,rs149733995,22,51238249,A,C,0.159780,0.873054,137930
11991346,rs574579322,22,51239584,G,C,-0.768483,0.442200,74004
11991347,rs535432390,22,51239586,T,G,-0.737201,0.461000,74004
11991348,rs3888401,22,51242557,A,T,-1.158893,0.246500,74004


### only need to merge MAF for wightman

In [124]:
wightman_maf = pd.DataFrame()
for chrom in range(1,23):
    print("start chr %d"%(chrom))
    maf_path = '/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/ukb_maf/ukb_mfi_chr' + str(chrom) +"_v3.txt"
    maf_info = pd.read_csv(maf_path, sep = '\t', names = ['UNIQID.A1A2', 'SNP','BP','A1','A2','MAF','NA','INFO'], usecols= ['SNP','A1','A2','MAF'])
    MAF=wightman[wightman.CHR == chrom].merge(maf_info, how = 'inner', on='SNP')
    wightman_maf = pd.concat([wightman_maf, MAF], ignore_index = True)


start chr 1
start chr 2
start chr 3
start chr 4
start chr 5
start chr 6
start chr 7
start chr 8
start chr 9
start chr 10
start chr 11
start chr 12
start chr 13
start chr 14
start chr 15
start chr 16
start chr 17
start chr 18
start chr 19
start chr 20
start chr 21
start chr 22


In [125]:
## Change the MAF for those that has A1,A2 filpped. (96 in total)
wightman_maf.loc[wightman_maf.A1_x != wightman_maf.A1_y,'MAF'] = 1 - wightman_maf.loc[wightman_maf.A1_x != wightman_maf.A1_y,'MAF']

In [127]:
wightman_maf = wightman_maf.drop(columns=["A1_y","A2_y"]).rename(columns= {'A1_x':'A1', 'A2_x':'A2'})
wightman_maf

,SNP,CHR,BP,A1,A2,Z,P,N,MAF
0,rs533499096,1,14860,G,T,-1.275874,0.2020,74004,0.000006
1,rs555296411,1,108506,C,T,0.269688,0.7874,74004,0.005777
2,rs62642118,1,108929,C,G,0.186567,0.8520,74004,0.008015
3,rs372054918,1,109503,G,A,-0.898723,0.3688,74004,0.001644
4,rs528106901,1,133855,C,T,-0.646895,0.5177,74004,0.000311
...,...,...,...,...,...,...,...,...,...
11725639,rs200189535,22,51235959,T,C,-0.060823,0.9515,74004,0.196421
11725640,rs534679861,22,51236137,C,T,0.741814,0.4582,74004,0.002108
11725641,rs3896457,22,51237063,T,C,-0.636884,0.5242,137930,0.292748
11725642,rs574579322,22,51239584,G,C,-0.768483,0.4422,74004,0.000015


## calculate beta for wightman
#### formula: Beta = z / sqrt(2MAF(1− MAF)(n + z^2))

In [128]:
wightman_maf["BETA"] = wightman_maf.Z/np.sqrt(2*wightman_maf.MAF*(1-wightman_maf.MAF)*wightman_maf.N + wightman_maf.Z ** 2 )

In [129]:
wightman_maf.to_csv('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/processed/wightman_fixed_beta.tsv',sep = '\t',index = False)